In [ ]:
import warnings

# Suppress the FutureWarnings related to is_sparse
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
import pandas as pd
import numpy as np

# Labes
L1 = 'label_1'
L2 = 'label_2'
L3 = 'label_3'
L4 = 'label_4'

LABELS = [L1,L2,L3,L4]
# Features
FEATURES = [f"feature_{i}" for i in range (1,769)]

In [ ]:
train_df = pd.read_csv("train.csv")
valid_df = pd.read_csv("valid.csv")
test_df = pd.read_csv("test.csv")

In [ ]:
print(f"train{train_df.shape},val{valid_df.shape},test{test_df.shape}")

In [ ]:
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Create subplots for each label
fig, axs = plt.subplots(2, 2, figsize=(14, 10))

for idx, label in enumerate(LABELS):
    ax = axs[idx // 2, idx % 2]  # Determines the position of the subplot

    # For label_2 (Age), specify the age range you want to visualize
    if label == L2:
        age_range = (0, 100)  # Adjust the range as needed
        filtered_df = train_df[(train_df[label] >= age_range[0]) & (train_df[label] <= age_range[1])]
        ax.hist(filtered_df[label], bins=20, range=age_range, edgecolor='k', color='green')
        ax.set_title(f'Distribution of {label} in the Range {age_range[0]} - {age_range[1]}')
        ax.set_xlabel(label)
        ax.set_ylabel('Count')
        ax.set_xticks(range(age_range[0], age_range[1] + 1, 5))  # Adjust the x-axis ticks as needed
        ax.grid(axis='y', linestyle='--', alpha=0.7)
    else:
        sns.countplot(x=label, data=train_df, ax=ax, color='green')  # Set the color to green
        ax.set_title(f'Distribution of {label}')
        ax.set_ylabel('Count')
        ax.set_xlabel(label)

plt.tight_layout()
plt.show()

In [ ]:
train_df[L2]

In [ ]:
from sklearn.preprocessing import StandardScaler
x_train = {}
x_valid = {}
y_train = {}
y_valid = {}
x_test = {}

In [ ]:
# Scaler the data
for target_label in LABELS:

  # Dropping missing values
  tr_df = train_df[train_df['label_2'].notna()] if target_label == 'label_2' else train_df
  vl_df = valid_df[valid_df['label_2'].notna()] if target_label == 'label_2' else valid_df

  scaler = StandardScaler()

  x_train[target_label] = pd.DataFrame(scaler.fit_transform(tr_df.drop(LABELS, axis = 1)), columns=FEATURES)
  y_train[target_label] = tr_df[target_label]

  x_valid[target_label] = pd.DataFrame(scaler.transform(vl_df.drop(LABELS, axis = 1)), columns=FEATURES)
  y_valid[target_label] = vl_df[target_label]

  # x_test[target_label] = pd.DataFrame(scaler.transform(test_df.drop(LABELS, axis = 1)), columns=FEATURES)

In [ ]:
print(f"train-L1 {len(x_train['label_1'])},val-L1 {len(x_valid['label_1'])}")
print(f"train-L2 {len(x_train['label_2'])},val-L2 {len(x_valid['label_2'])}")
print(f"train-L3 {len(x_train['label_3'])},val-L3 {len(x_valid['label_3'])}")
print(f"train-L4 {len(x_train['label_4'])},val-L4 {len(x_valid['label_4'])}")

In [ ]:
x_test = pd.DataFrame(scaler.transform(test_df.drop('ID', axis = 1)), columns=FEATURES)
id_test = test_df['ID']

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn import metrics

### Cross Validation Function

In [ ]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np

def k_fold_cross_validation(model, X, y, k=5):
    if model == 'svc':
        classifier = svm.SVC()
    elif model == 'kNN':
        classifier = KNeighborsClassifier(n_neighbors=k)
    elif model == 'NBay':
        classifier = MultinomialNB()
    elif model == 'RF':
        classifier = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
    elif model == 'xgb':
        # Encode class labels to start from 0 and be consecutive integers
        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(y)

        classifier = xgb.XGBClassifier(
            n_estimators=100,             # Number of boosting rounds (you can tune this)
            max_depth=3,                  # Maximum depth of each tree (you can tune this)
            learning_rate=0.1,            # Learning rate (you can tune this)
            random_state=42               # Random seed for reproducibility
        )

    # Create a KFold cross-validation object
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    # Convert pandas DataFrames to NumPy arrays
    X = X.values
    y = y.ravel()  # Ensure y is one-dimensional

    # Perform k-fold cross-validation
    scores = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
        X_train_fold, X_test_fold = X[train_idx], X[test_idx]
        y_train_fold, y_test_fold = y[train_idx], y[test_idx]

        # Ensure X_train_fold and X_test_fold are non-negative
        X_train_fold = np.maximum(X_train_fold, 0)
        X_test_fold = np.maximum(X_test_fold, 0)

        # Fit the classifier on the training fold
        classifier.fit(X_train_fold, y_train_fold)

        # Evaluate the classifier on the test fold
        fold_score = classifier.score(X_test_fold, y_test_fold)
        scores.append(fold_score)

        print(f"{model} - Fold {fold + 1}/{k}: Score = {fold_score:.4f}")

    print(f"{model} - Average Score: {np.mean(scores):.4f}")

# Example usage:
# k_fold_cross_validation('xgb', x_train_trans_L1, y_train[L1])


In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV, KFold

# Label_01
0.97 - svm


In [ ]:
x_train[L1].head()

In [ ]:
# clf = svm.SVC( class_weight='balanced')
# clf.fit(x_train[L1], y_train[L1])
# y_pred = clf.predict(x_valid[L1])

In [ ]:
# print("========================= Acuuracy Before Feature Selection Label 1 =========================")
# print(f"Accuracy: {metrics.accuracy_score(y_valid[L1], y_pred)}")
# print(f"Precision: {metrics.precision_score(y_valid[L1], y_pred, average='weighted')}")
# print(f"Recall: {metrics.recall_score(y_valid[L1], y_pred,average='weighted')}")

## Feature Selection + Model Selection

In [ ]:
pca = PCA(n_components = 0.97, svd_solver ='full')
pca.fit(x_train[L1])
x_train_trans_L1 = pd.DataFrame(pca.transform(x_train[L1]))
x_valid_trans_L1 = pd.DataFrame(pca.transform(x_valid[L1]))
x_test_trans_L1 = pd.DataFrame(pca.transform(x_test))

In [ ]:
x_train_trans_L1.shape

In [ ]:
# k_fold_cross_validation('svc', x_train_trans_L1, y_train[L1])
# k_fold_cross_validation('kNN',x_train_trans_L1, y_train[L1])
# k_fold_cross_validation('NBay',x_train_trans_L1, y_train[L1])
# k_fold_cross_validation('RF',x_train_trans_L1, y_train[L1])

In [ ]:
# clf = svm.SVC(class_weight='balanced')
# clf.fit(x_train_trans_L1, y_train[L1])
# y_pred_L1 = clf.predict(x_valid_trans_L1 )


In [ ]:
# print("========================= Acuuracy Before Hyper Parameter Tune Label 1 =========================")
# print(f"Accuracy: {metrics.accuracy_score(y_valid[L1], y_pred_L1)}")
# print(f"Precision: {metrics.precision_score(y_valid[L1], y_pred_L1, average='weighted')}")
# print(f"Recall: {metrics.recall_score(y_valid[L1], y_pred_L1,average='weighted')}")

## Hyper Parameter Tuning

In [ ]:
# Define the parameter grid for hyperparameter search
param_dist = {
    'C': [0.1,1,10],  # Regularization parameter
    'kernel': ['linear', 'rbf', 'poly'],  # Kernel type
    'degree': [1,2] , # Degree of the polynomial kernel
    'gamma': [0.001, 0.1,1]  # Kernel coefficient
}

k = 4  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Create an SVC classifier
clf = svm.SVC()

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_dist,
    n_iter=3,  # Number of random parameter combinations to try
    cv=kf,  # Number of cross-validation folds
    random_state=42,  # Set a random seed for reproducibility
    n_jobs=-1,  # Use all available CPU cores
    verbose=2
)

# Fit the RandomizedSearchCV to the data
random_search.fit(x_train_trans_L1, y_train[L1])

# Get the best estimator and its parameters
best_estimator = random_search.best_estimator_
best_params = random_search.best_params_

print("Best Parameters:", best_params)

# Now you can use the best_estimator for predictions
y_new_trans = best_estimator.predict(x_valid_trans_L1)

In [ ]:
y_test_L1 = best_estimator.predict(x_test_trans_L1)

In [ ]:
print("========================= Acuuracy After k fold Cross Validation and Hyper Parameter Tuning Label 1 =========================")
print(f"Accuracy: {metrics.accuracy_score(y_valid[L1], y_new_trans)}")
print(f"Precision: {metrics.precision_score(y_valid[L1], y_new_trans, average='weighted')}")
print(f"Recall: {metrics.recall_score(y_valid[L1], y_new_trans,average='weighted')}")

In [ ]:
# k_fold_cross_validation('svc', x_train_trans_L1, y_train[L1])
# k_fold_cross_validation('kNN', x_train_trans_L1, y_train[L1])

# Label_2
0.98 - kNN

In [ ]:
# clf = svm.SVC(class_weight='balanced')
# clf.fit(x_train[L2], y_train[L2])
# y_pred_L2 = clf.predict(x_valid[L2])

In [ ]:
# print("========================= Acuuracy Before Feature Selection Label 2 =========================")
# print(f"Accuracy: {metrics.accuracy_score(y_valid[L2], y_pred_L2)}")
# print(f"Precision: {metrics.precision_score(y_valid[L2], y_pred_L2, average='weighted')}")
# print(f"Recall: {metrics.recall_score(y_valid[L2], y_pred_L2,average='weighted')}")

## Feature Selection + Model Selection

In [ ]:
pca = PCA(n_components = 0.98, svd_solver ='full')
pca.fit(x_train[L2])
x_train_trans_L2 = pd.DataFrame(pca.transform(x_train[L2]))
x_valid_trans_L2 = pd.DataFrame(pca.transform(x_valid[L2]))
x_test_trans_L2 = pd.DataFrame(pca.transform(x_test))

In [ ]:
x_train_trans_L2.shape

In [ ]:
# k_fold_cross_validation('svc', x_train_trans_L2, y_train[L2])
# k_fold_cross_validation('kNN',x_train_trans_L2, y_train[L2])
# k_fold_cross_validation('NBay',x_train_trans_L2, y_train[L2])
# k_fold_cross_validation('RF',x_train_trans_L2, y_train[L2])

In [ ]:
# clf = svm.SVC(class_weight='balanced')
# clf.fit(x_train_trans_L2, y_train[L2])
# y_pred_L2 = clf.predict(x_valid_trans_L2)


In [ ]:
# print("========================= Acuuracy Before Hyper Parameter Tune Label 2 =========================")
# print(f"Accuracy: {metrics.accuracy_score(y_valid[L2], y_pred_L2)}")
# print(f"Precision: {metrics.precision_score(y_valid[L2], y_pred_L2, average='weighted')}")
# print(f"Recall: {metrics.recall_score(y_valid[L2], y_pred_L2,average='weighted')}")

## Hyper Parameter Tuning

In [ ]:
# Define the kNN classifier
knn = KNeighborsClassifier()

# Define a random search space for hyperparameters
param_dist = {
    'n_neighbors': list(range(1, 31)),   # Number of neighbors to consider
    'weights': ['uniform', 'distance'],  # Weighting scheme for neighbors
    'p': [1, 2],                        # Minkowski distance parameter (1 for Manhattan, 2 for Euclidean)
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithm for nearest neighbors
}

# Create a RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=knn, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)

# Perform the random search on the training data
random_search.fit(x_train_trans_L2, y_train[L2])

# Print the best hyperparameters and the corresponding accuracy score
print("Best Hyperparameters:", random_search.best_params_)
print("Best Accuracy Score:", random_search.best_score_)

# Get the best model with the best hyperparameters
best_knn = random_search.best_estimator_

In [ ]:
y_test_L2 = best_knn.predict(x_test_trans_L2)

In [ ]:
# print("========================= Acuuracy After k fold Cross Validation and Hyper Parameter Tuning Label 2 =========================")
# print(f"Accuracy: {metrics.accuracy_score(y_valid[L2], y_new_trans_L2)}")
# print(f"Precision: {metrics.precision_score(y_valid[L2], y_new_trans_L2, average='weighted')}")
# print(f"Recall: {metrics.recall_score(y_valid[L2], y_new_trans_L2,average='weighted')}")

# Label_03
0.95 - svm

In [ ]:
clf = svm.SVC(class_weight='balanced')
clf.fit(x_train[L3], y_train[L3])
y_pred_L3 = clf.predict(x_valid[L3])

In [ ]:
print("========================= Acuuracy Before Feature Selection Label 3 =========================")
print(f"Accuracy: {metrics.accuracy_score(y_valid[L3], y_pred_L3)}")
print(f"Precision: {metrics.precision_score(y_valid[L3], y_pred_L3, average='weighted')}")
print(f"Recall: {metrics.recall_score(y_valid[L3], y_pred_L3,average='weighted')}")

## Feature Selection + Model Selection

In [ ]:
pca = PCA(n_components = 0.95, svd_solver ='full')
pca.fit(x_train[L3])
x_train_trans_L3 = pd.DataFrame(pca.transform(x_train[L3]))
x_valid_trans_L3 = pd.DataFrame(pca.transform(x_valid[L3]))
x_test_trans_L3 = pd.DataFrame(pca.transform(x_test))

In [ ]:
x_train_trans_L3.shape

In [ ]:
# k_fold_cross_validation('svc', x_train_trans_L3, y_train[L3])
# k_fold_cross_validation('kNN',x_train_trans_L3, y_train[L3])
# k_fold_cross_validation('NBay',x_train_trans_L3, y_train[L3])
# k_fold_cross_validation('RF',x_train_trans_L3, y_train[L3])

In [ ]:
x_train_trans_L3.shape

In [ ]:
# clf = svm.SVC(class_weight='balanced')
# clf.fit(x_train_trans_L3, y_train[L3])
# y_pred_L3 = clf.predict(x_valid_trans_L3)


In [ ]:
# print("========================= Acuuracy Before Hyper Parameter Tune Label 3 =========================")
# print(f"Accuracy: {metrics.accuracy_score(y_valid[L3], y_pred_L3)}")
# print(f"Precision: {metrics.precision_score(y_valid[L3], y_pred_L3, average='weighted')}")
# print(f"Recall: {metrics.recall_score(y_valid[L3], y_pred_L3,average='weighted')}")

## Hyper Parameter Tuning

In [ ]:
# Define the parameter grid for hyperparameter search
param_dist = {
    'C': [0.1,1,10],  # Regularization parameter
    'kernel': [ 'linear','rbf', 'poly'],  # Kernel type
    'degree': [1,2] , # Degree of the polynomial kernel
    'gamma': [0.001, 0.1,1]  # Kernel coefficient
}

k = 5  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Create an SVC classifier
clf = svm.SVC()

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_dist,
    n_iter=3,  # Number of random parameter combinations to try
    cv=kf,  # Number of cross-validation folds
    random_state=42,  # Set a random seed for reproducibility
    n_jobs=-1,  # Use all available CPU cores
    verbose=2
)

# Fit the RandomizedSearchCV to the data
random_search.fit(x_train_trans_L3, y_train[L3])

# Get the best estimator and its parameters
best_estimator = random_search.best_estimator_
best_params = random_search.best_params_

print("Best Parameters:", best_params)

# Now you can use the best_estimator for predictions
y_new_trans_L3 = best_estimator.predict(x_valid_trans_L3)

In [ ]:
y_test_L3 = best_estimator.predict(x_test_trans_L3)

In [ ]:
print("========================= Acuuracy After k fold Cross Validation and Hyper Parameter Tuning Label 3 =========================")
print(f"Accuracy: {metrics.accuracy_score(y_valid[L3], y_new_trans_L3)}")
print(f"Precision: {metrics.precision_score(y_valid[L3], y_new_trans_L3, average='weighted')}")
print(f"Recall: {metrics.recall_score(y_valid[L3], y_new_trans_L3,average='weighted')}")

# Label_04
0.97 - kNN

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
# pip install -U imbalanced-learn

In [ ]:
y_train[L4]

In [ ]:
# clf = svm.SVC()
# clf.fit(x_train[L4], y_train[L4])
# y_pred_L4 = clf.predict(x_valid[L4])

In [ ]:
# print("========================= Acuuracy Before Feature Selection Label 3 =========================")
# print(f"Accuracy: {metrics.accuracy_score(y_valid[L4], y_pred_L4)}")
# print(f"Precision: {metrics.precision_score(y_valid[L4], y_pred_L4, average='weighted')}")
# print(f"Recall: {metrics.recall_score(y_valid[L4], y_pred_L4,average='weighted')}")

## Feature Selection + Model Selection

In [ ]:
pca = PCA(n_components = 0.97, svd_solver ='full')
pca.fit(x_train[L4])
x_train_trans_L4 = pd.DataFrame(pca.transform(x_train[L4]))
x_valid_trans_L4 = pd.DataFrame(pca.transform(x_valid[L4]))
x_test_trans_L4 = pd.DataFrame(pca.transform(x_test))

In [ ]:
x_train_trans_L4.shape

In [ ]:
# k_fold_cross_validation('svc', x_train_trans_L4, y_train[L4])
# k_fold_cross_validation('kNN',x_train_trans_L4, y_train[L4])
# k_fold_cross_validation('NBay',x_train_trans_L4, y_train[L4])
# k_fold_cross_validation('RF',x_train_trans_L4, y_train[L4])

In [ ]:
# clf = svm.SVC(class_weight='balanced')
# clf.fit(x_train_trans_L4, y_train[L4])
# y_pred_L4 = clf.predict(x_valid_trans_L4)


In [ ]:
# print("========================= Acuuracy Before Hyper Parameter Tune Label 3 =========================")
# print(f"Accuracy: {metrics.accuracy_score(y_valid[L4], y_pred_L4)}")
# print(f"Precision: {metrics.precision_score(y_valid[L4], y_pred_L4, average='weighted')}")
# print(f"Recall: {metrics.recall_score(y_valid[L4], y_pred_L4,average='weighted')}")

## Model Selection

In [ ]:
# k_fold_cross_validation('svc', x_train_trans_L4, y_train[L4])
# k_fold_cross_validation('kNN', x_train_trans_L4, y_train[L4])

## Hyper Parameter Tuning

In [ ]:
# # Define the kNN classifier
knn = KNeighborsClassifier()

# Define a random search space for hyperparameters
param_dist = {
    'n_neighbors': list(range(1, 31)),   # Number of neighbors to consider
    'weights': ['uniform', 'distance'],  # Weighting scheme for neighbors
    'p': [1, 2],                        # Minkowski distance parameter (1 for Manhattan, 2 for Euclidean)
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithm for nearest neighbors
}

# Create a RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=knn, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)

# Perform the random search on the training data
random_search.fit(x_train_trans_L4, y_train[L4])

# Print the best hyperparameters and the corresponding accuracy score
print("Best Hyperparameters:", random_search.best_params_)
print("Best Accuracy Score:", random_search.best_score_)

# Get the best model with the best hyperparameters
best_knn_L4 = random_search.best_estimator_

In [ ]:
y_test_L4 = best_knn_L4.predict(x_test_trans_L4)

In [ ]:
# print("========================= Acuuracy After k fold Cross Validation and Hyper Parameter Tuning Label 3 =========================")
# print(f"Accuracy: {metrics.accuracy_score(y_valid[L4], y_new_trans_L4)}")
# print(f"Precision: {metrics.precision_score(y_valid[L4], y_new_trans_L4, average='weighted')}")
# print(f"Recall: {metrics.recall_score(y_valid[L4], y_new_trans_L4,average='weighted')}")

# Result Writing

In [ ]:
output_df = pd.DataFrame({
    'ID': id_test,
    'label_1': y_test_L1,
    'label_2': y_test_L2,
    'label_3': y_test_L3,
    'label_4': y_test_L4
})

In [ ]:
# Save the DataFrame to the specified CSV file path
output_df.to_csv(f"output/UOM_190394R.csv", index=False)